Position Manager
================
The PositionManager is the trade repository for completed trades (which can include canceled or other reasons than filled). It
maintains current positions, trade blotter, PnL and is the entry point for metrics calculation.

In [1]:
# Import packages and initialize the PositionManager

import puma as tw
import pandas as pd

import database.symbol as symboldb
import data as datalib
from tomahawk import position_manager, OrderManager

oms = OrderManager('test_unit', None)
pm = position_manager.PositionManager('test_unit', oms, None)

Trade Entry
-----------

In [2]:
# Enter a trade from the trade details

pm.enter_trade('orig-id', 'strategy-id', pd.Timestamp('2010-01-05 13:04:00', tz='America/New_York'), 'stock', 'test.sym.9', 'buy', 100, 50)

# View the positions DataFrame

print(pm.positions_df)

('strategy_id', 'product_type', 'symbol')      current_position    start_position    net_quantity    buy_quantity    sell_quantity    buy_avg_price    sell_avg_price    buy_pnl    sell_pnl    trade_pnl    position_pnl    gross_pnl    commission    net_pnl  prior_close_price    current_price
-------------------------------------------  ------------------  ----------------  --------------  --------------  ---------------  ---------------  ----------------  ---------  ----------  -----------  --------------  -----------  ------------  ---------  -------------------  ---------------
('strategy-id', 'stock', 'test.sym.9')                      100                 0             100             100                0               50                 0          0           0            0               0            0             0          0


In [3]:
# Enter a trade from Order object and view the positions DataFrame

order = tw.Order('o01', 'orig-id', '123-456', 'strategy-id', 'stock', 'test.sym.9', 'buy', 100, 'LIMIT', price=100)
order.state = 'FILLED'
order.add_fill(1001, pd.Timestamp('2016-08-01 12:13:14', tz='America/New_York'), pd.Timestamp('2016-01-01 09:30:00', tz='America/New_York'), 100, 100, -1.0)
pm.enter_trade_from_order(order)

pm.positions_df.show()

('strategy_id', 'product_type', 'symbol')      current_position    start_position    net_quantity    buy_quantity    sell_quantity    buy_avg_price    sell_avg_price    buy_pnl    sell_pnl    trade_pnl    position_pnl    gross_pnl    commission    net_pnl  prior_close_price    current_price
-------------------------------------------  ------------------  ----------------  --------------  --------------  ---------------  ---------------  ----------------  ---------  ----------  -----------  --------------  -----------  ------------  ---------  -------------------  ---------------
('strategy-id', 'stock', 'test.sym.9')                      200                 0             200             200                0               75                 0          0           0            0               0            0            -1          0


Order Entry from OrderManager
-----------------------------

In [4]:
# create a new order and insert into OrderManager
order = tw.Order('123-456', 'manual_id', '002', 'strat-id', 'stock', 'test.sym.10', 'sell', 200, 'LIMIT', price=55.5)
oms.new_order(order)

In [5]:
# change state to FILLED
order.add_fill(1002, pd.Timestamp('2016-08-01 12:13:14', tz='America/New_York'), pd.Timestamp('2016-01-01 09:30:00', tz='America/New_York'), 200, 55.5, -2.0)
oms.change_state(order, 'FILLED')
oms.set_booked(order, False)
oms.closed_orders_df().show()

# Note that the booked state is False

  index  product_type    symbol       strategy_id    Order__uuid                           CREATED                           originator_uuid    state    closed    booked      fill_quantity  type    broker_order_id      strategy_uuid  portfolio_uuid    event_type    create_timestamp                  details          exchange_order_id    buy_sell      commission    quantity  FILLED                            originator_id    portfolio_id      fill_price
-------  --------------  -----------  -------------  ------------------------------------  --------------------------------  -----------------  -------  --------  --------  ---------------  ------  -----------------  ---------------  ----------------  ------------  --------------------------------  ---------------  -------------------  ----------  ------------  ----------  --------------------------------  ---------------  --------------  ------------
      0  stock           test.sym.10  strat-id       280bd50c-8ccf-4e1e-b6c6-a54d8d87222

In [6]:
# Use PositionManager to book the trade
pm.book_fills()

print(pm.positions_df)

('strategy_id', 'product_type', 'symbol')      current_position    start_position    net_quantity    buy_quantity    sell_quantity    buy_avg_price    sell_avg_price    buy_pnl    sell_pnl    trade_pnl    position_pnl    gross_pnl    commission    net_pnl  prior_close_price    current_price
-------------------------------------------  ------------------  ----------------  --------------  --------------  ---------------  ---------------  ----------------  ---------  ----------  -----------  --------------  -----------  ------------  ---------  -------------------  ---------------
('strat-id', 'stock', 'test.sym.10')                       -200                 0            -200               0              200                0              55.5          0           0            0               0            0            -2          0
('strategy-id', 'stock', 'test.sym.9')                      200                 0             200             200                0               75           

PnL Calculations
----------------

In [7]:
# A MarketDataManager must be setup and attached
db_credentials = credentials('test', 'localhost', 'db_')
seng = symboldb.symbol_engine('stock', **credentials('test', 'localhost'))
datafeed = datalib.SymbolDBDataFeed({'stock': seng}, source='test_source_02')
lmds = datalib.LiveDataManager(datafeed, **db_credentials)
hmds = datalib.LiveDataManager(datafeed, **db_credentials)
mdm = datalib.MarketDataManager(hmds, lmds)

pm.setup_market_data(mdm)

In [8]:
# update the market data manager
mdm.bartime = '2010-01-05 13:05:00'
mdm.update('stock', '1min')

# Update and run the pnl calculations
pm.update_pnl()
pm.positions_df.show()

('strategy_id', 'product_type', 'symbol')      current_position    start_position    net_quantity    buy_quantity    sell_quantity    buy_avg_price    sell_avg_price    buy_pnl    sell_pnl    trade_pnl    position_pnl    gross_pnl    commission    net_pnl    prior_close_price    current_price
-------------------------------------------  ------------------  ----------------  --------------  --------------  ---------------  ---------------  ----------------  ---------  ----------  -----------  --------------  -----------  ------------  ---------  -------------------  ---------------
('strat-id', 'stock', 'test.sym.10')                       -200                 0            -200               0              200                0              55.5          0        1198         1198            -476          722            -2        720                49.51            51.89
('strategy-id', 'stock', 'test.sym.9')                      200                 0             200             200     

Get Value
---------
Use the get_value() method to extract a single cells value from the PositionManager

In [9]:
pm.get_value('strategy-id', 'stock', 'test.sym.9', 'current_position')

200